In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import utils
import vgg19
images = tf.placeholder("float", [1, 224, 224, 3])
vgg = vgg19.Vgg19()
vgg.build(images) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\kggab\Documents\project\vgg19.npy
npy file loaded
build model started
build model finished: 3s


In [82]:
import copy
content_img = utils.load_image("./images/columbia2.JPG")
style_img = utils.load_image("./images/Ukiyo.JPG")
content_img = content_img.reshape(1, 224, 224, 3)
style_img = style_img.reshape(1, 224, 224, 3)
output = copy.copy(style_img)
data = {'conv1_1' : vgg.conv1_1,
        'conv2_1' : vgg.conv2_1,
        'conv3_1' : vgg.conv3_1,
        'conv4_1' : vgg.conv4_1,
        'conv5_1' : vgg.conv5_1}
#feed_dict = {images: content_img}
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())
#    sess.run(output, feed_dict = feed_dict)

In [75]:
import numpy as np
def get_cont_loss_layer(output, content, layer):
    content_layers = layer.split(',')
    losses = []
    for l in content_layers:
        vgg_l = data[l]
        losses.append(content_loss(output, content, vgg_l))
    return losses

def content_loss(output, content, layer):
    with tf.Session() as sess:
        F = sess.run(layer, {images: content_img})
        P = sess.run(layer, {images: output})
    shape = F.shape
    m,n = shape[1]*shape[2], shape[3]
    F = F.reshape(m,n)
    P = P.reshape(m,n)
    loss = 0.5 * tf.reduce_sum(tf.squared_difference(F, P))
    return loss
        
content_loss = get_cont_loss_layer(output, content_img, "conv1_1,conv2_1")
print(content_loss)

[<tf.Tensor 'mul_31:0' shape=() dtype=float32>, <tf.Tensor 'mul_32:0' shape=() dtype=float32>]


In [77]:
def get_style_loss_layer(output, style, layer):
    style_layers = layer.split(',')
    losses = []
    for l in style_layers:
        vgg_l = data[l]
        losses.append(style_loss(output, style, vgg_l))
    return losses

def gram_matrix(image, vgg_l):
    with tf.Session() as sess:
        conv1_1 = sess.run(vgg_l, {images: image})
    shape = conv1_1.shape
    m,n = shape[1]*shape[2], shape[3]
    vect_layer = conv1_1.reshape(m,n)
    gram = tf.matmul(vect_layer, vect_layer, transpose_a = True)
    return gram,m,n

def style_loss(output, style, vgg_l):
    A,m,n = gram_matrix(style, vgg_l)
    G,m,n = gram_matrix(output, vgg_l)
    loss = 1/(4*n^2*m^2) * tf.reduce_sum(tf.squared_difference(A, G))
    return loss

style_loss = get_style_loss_layer(output, style_img, "conv1_1,conv2_1,conv3_1,conv4_1,conv5_1")
print(style_loss)

[<tf.Tensor 'mul_33:0' shape=() dtype=float32>, <tf.Tensor 'mul_34:0' shape=() dtype=float32>, <tf.Tensor 'mul_35:0' shape=() dtype=float32>, <tf.Tensor 'mul_36:0' shape=() dtype=float32>, <tf.Tensor 'mul_37:0' shape=() dtype=float32>]


In [84]:
style_weights = 1/5
lr = 2.0
beta = 1000
alpha = 1
total_loss = beta * np.sum(content_loss) + alpha * style_weights * np.sum(style_loss)
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(total_loss)
iterations = 5

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        sess.run(train_step)
        print("loss:{}".format(sess.run(total_loss)))
    

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(1, 224, 224, 3) dtype=float32_ref>", "<tf.Variable 'Variable_1:0' shape=(1, 224, 224, 3) dtype=float32_ref>"] and loss Tensor("add_6:0", shape=(), dtype=float32).

In [71]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imgplot = plt.imshow(img)


NameError: name 'img' is not defined